# Create lookup table between SUSO and OSSE

Takes in: 

- Data on SUSO participants
- OSSE student attributes to use for matching

What it does: exact and fuzzy matching on the basis of name, DOB, and school attended

Outputs: lookup table to get usi's (OSSE identifiers) for students in SUSO sample

In [1]:
import re
from pathlib import Path

import numpy as np
import pandas as pd
import recordlinkage
from IPython.core.interactiveshell import InteractiveShell

from suso.utils import PICKLE_PROTOCOL, here

InteractiveShell.ast_node_interactivity = "all"

In [2]:
DATA_DIR = here("data")

# 1: Load data



## 1.1 Load SUSO outreach data

In [3]:
df = pd.read_csv(DATA_DIR / "data_for_analysis.csv")
bounced = pd.read_csv(DATA_DIR / "returned_letters.csv")
bounced = bounced.drop_duplicates("CLID", keep="first")

In [4]:
suso = pd.read_csv(DATA_DIR / "suso_participants.csv")
suso["in_suso"] = True

In [5]:
df_suso = df.merge(
    suso[["CLID", "DateEngaged", "DateNotEngaged", "in_suso", "DateOfBirthNew"]],
    how="left",
    on="CLID",
)

df_suso.to_csv(DATA_DIR / "df_suso_merged.csv")

## 1.3 Load credentials and pull student identifiers from OSSE database

In [6]:
def read_attendance_data(path: Path) -> pd.DataFrame:
    return (
        pd.read_parquet(
            path,
            columns=[
                "usi",
                "firstname",
                "lastname",
                "StudentLocalID",
                "dateofbirth",
                "GradeLevel",
                "gender",
                "race",
                "Enr_SchoolID",
                "Enr_SchoolName",
                "AttendanceDate",
            ],
        )
        .sort_values(by=["usi", "AttendanceDate"])
        .drop_duplicates("usi", keep="first")
        .drop(columns="AttendanceDate")
    )

In [7]:
entities_public = read_attendance_data(
    DATA_DIR / "SY1718_DCPS_Attendance_Data_cleaned.parquet"
)
entities_public.to_pickle(DATA_DIR / "entities_public.pkl", protocol=PICKLE_PROTOCOL)

In [8]:
entities_charter = read_attendance_data(
    DATA_DIR / "SY1718_Charter_Sector_Attendance_cleaned.parquet"
)
entities_charter.to_pickle(DATA_DIR / "entities_charter.pkl", protocol=PICKLE_PROTOCOL)

In [9]:
## merge charter and public school into one df
entities = pd.concat([entities_public, entities_charter])
entities.to_pickle(DATA_DIR / "entities_charterpublic.pkl", protocol=PICKLE_PROTOCOL)

In [10]:
## dx
print("Total number of rows is: " + str(entities.shape[0]))
print("Number of unique student identifiers is: " + str(len(entities.usi.unique())))

Total number of rows is: 82781
Number of unique student identifiers is: 81701


# 2: identify exact matches using name and DOB

In [11]:
osse_matchcols = [
    "firstname",
    "lastname",
    "dateofbirth",
    "GradeLevel",
    "Enr_SchoolName",
]
suso_matchcols = [
    "FName",
    "FamilyNames",
    "LName",
    "guardian_firstname",
    "guardian_lastname",
    "school_name",
    "DateOfBirthNew",
    "grade",
]

In [12]:
%%capture

df_suso[suso_matchcols].head()

In [13]:
%%capture

entities[osse_matchcols].head()

##  2.1: convert names and birth date to same format in each dataset

In [14]:
## capitalize names in suso and remove whitespace
df_suso["studentfirstname_suso"] = df_suso.FName.str.upper().str.strip()
df_suso["studentlastname_suso"] = df_suso.LName.str.upper().str.strip()

In [15]:
## remove whitespace from osse names
entities["studentfirstname_osse"] = entities.firstname.str.strip()
entities["studentlastname_osse"] = entities.lastname.str.strip()

In [16]:
## dob as datetime in each
df_suso["dob_suso"] = pd.to_datetime(df_suso.DateOfBirthNew)
entities["dob_osse"] = pd.to_datetime(entities.dateofbirth)

In [17]:
## create single identifier
df_suso["name_dob"] = (
    df_suso.studentfirstname_suso
    + "_"
    + df_suso.studentlastname_suso
    + "_"
    + df_suso.dob_suso.astype(str)
)

In [18]:
entities["name_dob"] = (
    entities.studentfirstname_osse
    + "_"
    + entities.studentlastname_osse
    + "_"
    + entities.dob_osse.astype(str)
)

##  2.2: look for exact matches

In [19]:
unique_suso_namedob = set(df_suso.name_dob)
unique_osse_namedob = set(entities.name_dob)

In [20]:
exact_match = unique_suso_namedob.intersection(unique_osse_namedob)

In [21]:
## Update
print(
    "Able to merge " + str(len(exact_match)) + " students with exact name and DOB match"
)
print(
    "This represents "
    + str(round(len(exact_match) / len(unique_suso_namedob), 3) * 100)
    + " percent of the SUSO sample"
)

Able to merge 1161 students with exact name and DOB match
This represents 81.39999999999999 percent of the SUSO sample


##  2.3: construct lookup table with those exact matches and then subset to unmatched students

In [22]:
%%capture

matched_suso = df_suso.copy()
matched_suso["suso_id"] = matched_suso.student_id
matched_suso_tomerge = matched_suso[["suso_id", "name_dob"]]
matched_suso_tomerge.head()
suso_lookup_exactmatch = pd.merge(
    matched_suso_tomerge,
    entities[["usi", "name_dob"]].drop_duplicates(),
    on="name_dob",
    how="inner",
)

In [23]:
## subset to students with no exact match
suso_noexactmatch = (
    matched_suso.loc[~matched_suso.suso_id.isin(suso_lookup_exactmatch.suso_id)]
    .copy()
    .reset_index()
)


osse_noexactmatch = (
    entities.loc[~entities.usi.isin(suso_lookup_exactmatch.usi)].copy().reset_index()
)

##  2.4: use fuzzy string matching to try to match next round

### Convert school names to similar format to have another fuzzy matching variable

In [24]:
## source: https://stackoverflow.com/questions/6116978/how-to-replace-multiple-substrings-of-a-string
def multiple_replace(string, rep_dict):
    pattern = re.compile(
        "|".join([re.escape(k) for k in sorted(rep_dict, key=len, reverse=True)]),
        flags=re.DOTALL,
    )
    return pattern.sub(lambda x: rep_dict[x.group(0)], string)

In [25]:
school_replace_dict = {
    "Elementary School": "ES",
    "Education Campus": "EC",
    "Middle School": "MS",
    "High School": "HS",
}

In [26]:
## convert school name to similar format
osse_noexactmatch["schoolname_tomerge_1"] = osse_noexactmatch.Enr_SchoolName.astype(
    str
).apply(multiple_replace, rep_dict=school_replace_dict)
osse_noexactmatch[
    "schoolname_tomerge_osse"
] = osse_noexactmatch.schoolname_tomerge_1.str.upper()

In [27]:
suso_noexactmatch["schoolname_tomerge_suso"] = suso_noexactmatch.school_name.str.upper()

### Use fuzzy string matching on student and school name to match

#### Create matched pairs

In [28]:
def fm_createpairs(data_1, data_2, blocking_variable, matching_dictionary):

    ## first initialize an indexer
    indexer = recordlinkage.Index()
    indexer.block(blocking_variable)

    ## then, feed the indexer the two datasets (both must have the blocking variable)
    candidate_links = indexer.index(data_1, data_2)

    ## then, start comparison
    c = recordlinkage.Compare()

    ## iterate through variables to do matching on
    for key, value in matching_dictionary.items():
        c.string(value[0], value[1], method=value[2], threshold=value[3])

    ## use those variables to create feature vectors
    feature_vectors = c.compute(candidate_links, data_1, data_2)

    ## now, uses unsupervised clustering algorithm to create matches
    ## could generalize function to include other algorithms
    ecm = recordlinkage.ECMClassifier()
    predicted_matches_ecm = list(ecm.fit_predict(feature_vectors))
    return predicted_matches_ecm

In [29]:
## create common key
suso_noexactmatch["dob"] = suso_noexactmatch.dob_suso
osse_noexactmatch["dob"] = osse_noexactmatch.dob_osse

In [30]:
## apply function
matching_dictionary = {
    "first_var": ["studentfirstname_suso", "studentfirstname_osse", "jarowinkler", 0.8],
    "second_var": ["studentlastname_suso", "studentlastname_osse", "jarowinkler", 0.8],
    "third_var": [
        "schoolname_tomerge_suso",
        "schoolname_tomerge_osse",
        "jarowinkler",
        0.8,
    ],
}
matched_pairs = fm_createpairs(
    suso_noexactmatch, osse_noexactmatch, "dob", matching_dictionary
)

#### Retrieve matched pairs

In [31]:
suso_indices = [x[0] for x in matched_pairs]
osse_indices = [x[1] for x in matched_pairs]

In [32]:
suso_osse_matchdf = pd.DataFrame(
    {"suso_indices": suso_indices, "osse_indices": osse_indices}
).sort_values(by="suso_indices")

In [33]:
suso_noexactmatch["suso_indices"] = suso_noexactmatch.index
osse_noexactmatch["osse_indices"] = osse_noexactmatch.index

In [34]:
## merge cols
suso_mergecols = [
    "suso_indices",
    "suso_id",
    "studentfirstname_suso",
    "studentlastname_suso",
    "schoolname_tomerge_suso",
    "dob_suso",
    "name_dob",
]
osse_mergecols = [
    "osse_indices",
    "usi",
    "StudentLocalID",
    "studentfirstname_osse",
    "studentlastname_osse",
    "schoolname_tomerge_osse",
    "dob_osse",
]

In [35]:
## merge relevant records
suso_osse_matchdf_withsusoid = suso_osse_matchdf.merge(
    suso_noexactmatch[suso_mergecols], how="left", on="suso_indices"
)
suso_osse_matchdf_withboth = suso_osse_matchdf_withsusoid.merge(
    osse_noexactmatch[osse_mergecols], how="left", on="osse_indices"
)

In [36]:
## for duplicate ids (multiple matches in OSSE, take first match (highestprob))
suso_ossematchdf_withboth_dedup = suso_osse_matchdf_withboth.drop_duplicates(
    subset=["suso_id"], keep="first"
)

In [37]:
## write to data to manually inspect
suso_ossematchdf_withboth_dedup.to_csv(
    DATA_DIR / "suso_osse_fuzzymatches.csv", index=False
)

## 2.5 Visually inspect random sample and then subset to final vars

In [38]:
%%capture

suso_ossematchdf_withboth_dedup.sample(frac=0.1)

In [39]:
%%capture

suso_lookup_fuzzymatch = suso_ossematchdf_withboth_dedup[["suso_id", "name_dob", "usi"]]

suso_lookup_fuzzymatch.head()
print(
    "Able to merge "
    + str(suso_lookup_fuzzymatch.shape[0])
    + " additional students with exact DOB and fuzzy name/school name match"
)

## 2.6  Merge lookup table with pairs from first round

In [40]:
## before merging, add indicator for whether it was an exact or
## fuzzy match
suso_lookup_exactmatch["type_of_match"] = "exact"
suso_lookup_fuzzymatch["type_of_match"] = "fuzzy"

/var/folders/_8/wxdcgmrd7b931622mzvpxn6c0000gq/T/ipykernel_96224/4207313620.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suso_lookup_fuzzymatch["type_of_match"] = "fuzzy"


In [41]:
suso_exactandfuzzy = pd.concat([suso_lookup_exactmatch, suso_lookup_fuzzymatch])

In [42]:
## restrict to students with valid treatment status (some students were referred to suso but never got assigned treatment)
valid_tx_suso = df_suso.student_id[df_suso.is_treatment.notnull()]

suso_exactandfuzzy_validtx = suso_exactandfuzzy[
    suso_exactandfuzzy.suso_id.isin(valid_tx_suso)
].copy()
suso_exactandfuzzy_validtx.shape

(1198, 4)

In [43]:
print(
    "Able to merge "
    + str(suso_exactandfuzzy.shape[0])
    + " referred students after 1) exact match, and 2) fuzzy match with high threshold"
)
print(
    "This represents "
    + str(round(suso_exactandfuzzy.shape[0] / len(unique_suso_namedob), 3) * 100)
    + " percent of those referred"
)

print(
    "Able to merge "
    + str(suso_exactandfuzzy_validtx.shape[0])
    + " students w/ valid tx status after 1) exact match, and 2) fuzzy match with high threshold"
)
print(
    "This represents "
    + str(round(suso_exactandfuzzy_validtx.shape[0] / len(valid_tx_suso), 3) * 100)
    + " percent of those with valid tx status"
)

Able to merge 1323 referred students after 1) exact match, and 2) fuzzy match with high threshold
This represents 92.80000000000001 percent of those referred
Able to merge 1198 students w/ valid tx status after 1) exact match, and 2) fuzzy match with high threshold
This represents 94.0 percent of those with valid tx status


In [44]:
## write those referred to suso
suso_exactandfuzzy.to_pickle(
    DATA_DIR / "suso_osse_lookup.pkl", protocol=PICKLE_PROTOCOL
)